# Praca domowa 3
Celem tej pracy domowej jest wytrenowanie trzech klasyfikatorów i wybranie z pośród nich najlepszego na podstawie trzech wybranych miar jakości.

## Zbiór danych 
Korzystamy z danych o pogodzie w Australii. Na początku podzielimy dane na zbiór testowy i treningowy.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support
import random

import warnings
warnings.filterwarnings('ignore')
np.random.seed(13)
random.seed(13)

In [2]:
df = pd.read_csv('../australia.csv')

In [3]:
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
1,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
2,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0
3,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,0
4,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,0


In [4]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
count,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000
mean,13.464770,24.219206,2.130397,5.503135,7.735626,40.877366,15.667228,19.786778,65.874123,49.601985,1017.239505,1014.795580,4.241705,4.326515,18.204961,22.710333,0.220879,0.220259
std,6.416689,6.970676,7.014822,3.696282,3.758153,13.335232,8.317005,8.510180,18.513289,20.197040,6.909357,6.870892,2.797162,2.647251,6.567991,6.836543,0.414843,0.414425
min,-6.700000,4.100000,0.000000,0.000000,0.000000,9.000000,2.000000,2.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-0.700000,3.700000,0.000000,0.000000
25%,8.600000,18.700000,0.000000,2.800000,5.000000,31.000000,9.000000,13.000000,55.000000,35.000000,1012.700000,1010.100000,1.000000,2.000000,13.100000,17.400000,0.000000,0.000000
50%,13.200000,23.900000,0.000000,5.000000,8.600000,39.000000,15.000000,19.000000,67.000000,50.000000,1017.200000,1014.700000,5.000000,5.000000,17.800000,22.400000,0.000000,0.000000
75%,18.400000,29.700000,0.600000,7.400000,10.700000,48.000000,20.000000,26.000000,79.000000,63.000000,1021.800000,1019.400000,7.000000,7.000000,23.300000,27.900000,0.000000,0.000000
max,31.400000,48.100000,206.200000,81.200000,14.500000,124.000000,67.000000,76.000000,100.000000,100.000000,1040.400000,1038.900000,8.000000,9.000000,39.400000,46.100000,1.000000,1.000000


In [5]:
print("Procent próbek, gdzie następnego dnia nie pada deszcz")
df[df["RainTomorrow"]== 0].shape[0]/df.shape[0] * 100

Procent próbek, gdzie następnego dnia nie pada deszcz


77.974122651542

In [6]:
X = df.drop(columns=["RainTomorrow"])
y = df.RainTomorrow
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [7]:
y_train.shape[0]

45136

In [8]:
x_test.shape[0]

11284

## Trenowanie modeli 

### Linear regresion
- Toleranja - Kryterium zatrzymania uczenia. Ustawiamy na 0.0000001
- Max_iter - maksymalna liczba iteracji. Ustawaimy na 600
### Decision Tree
- min_samples_split - minimalna liczba próbek potrzebna do rozdizelenia wewnętrznego liścia. Zmieniamy z 2 na 3.
- min_samples_leaf - minimalna liczba próbek w liściu. Do podziału dojedzie tylko wtedy gdy po podziale zostanie co najmniej min_samples_leaf próbek w liściu. Ustawiamy z 1 na 2

### SVC
- C - parametr regularyzacji. Własnymi słowami odpowiada jak bardzo tolerujemy złe wskazania. Ustawiamy na 5
- Max_iter - maksymalna liczba iteracji. Ustawaimy na 800

In [9]:
clf_LogReg = LogisticRegression(random_state=0, max_iter = 600, tol = 0.0000001).fit(x_train, y_train)
clf_Tree = DecisionTreeClassifier(random_state=0, min_samples_split = 3, min_samples_leaf = 2).fit(x_train, y_train)
clf_SVC = LinearSVC(C = 2, random_state=0, max_iter = 800).fit(x_train, y_train)

## Porównanie wyników modeli

In [10]:
models = [clf_LogReg, clf_Tree, clf_SVC]

for model in models:
    print("Model: ", type(model).__name__ ,"| mean accuracy=", round(model.score(x_test, y_test), 5),
          ", precision =", round(precision_recall_fscore_support(model.predict(x_test), y_test, average='binary')[0], 5), 
          ", recall =", round(precision_recall_fscore_support(model.predict(x_test), y_test, average='binary')[1], 5),
          ", f1 =", round(precision_recall_fscore_support(model.predict(x_test), y_test, average='binary')[2], 5))

Model:  LogisticRegression | mean accuracy= 0.8489 , precision = 0.51045 , recall = 0.73606 , f1 = 0.60284
Model:  DecisionTreeClassifier | mean accuracy= 0.80548 , precision = 0.51677 , recall = 0.57456 , f1 = 0.54413
Model:  LinearSVC | mean accuracy= 0.77792 , precision = 0.01183 , recall = 0.96774 , f1 = 0.02338


Najlepszym modelem z powyższych jest zdecydowanie regresja logistyczna. Oprócz najwyższej dokladności ma również najwyższy wynik f1. Model oparty na drzewie decyzyjnym ma gorsze wszystkie metryki od modelu regresji. Model oparty na SVC wypadł zdecydowanie najgorzej. Jego dokładność jest porównywalna z dokładnością wpisania w próbce testowej wszędzie zer, gdyż mniej więcej taki procent próbek negatywnych jest w całym zbiorze. O tym, że model troche na ślepo przypisuje zera świadczy też bardzo wyzoki recall i bardzo niska precyzja. Model SVC w tym przypadku jest bezurzyteczny 